# Basic Gates Card Game

Learn about basic gates with a friend by trying to control the central qubit.  
Gain points by keeping the central Qubit 1, lose a point every time it's zero.

In [62]:
import qsharp
from random import randint
import time
from SimpleCardGame import PlayAndMeasure

In [173]:
# PlayAndMeasure.simulate(state=False, gate="H")

In [161]:
class Card:
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return self.name

In [169]:
class Game:
    def __init__(self, card_frequencies, cards_per_player, player_names):
        self.card_frequencies = card_frequencies
        self.cards_per_player = cards_per_player    
        self.player_names = player_names

        self.cards = []
        self.players = []
        self.state = False
        self.player_turn = 0
        self.player_updates = {"win point": "You earned a point!",
                              "lose point": "You lost a point!",
                              "win game": "You won the game!",
                              "lose game": "You lost the game!"}
        self.game_over = False

        self.initialize()

    def initialize(self):
        self.initialize_cards()
        self.initialize_players()
        self.deal_cards()
        
    def initialize_players(self):
        score = int(round(100/len(self.player_names)))
        for name in self.player_names:
            player = Player(name)
            player.score = score
            self.players.append(player)
            
        
    def initialize_cards(self):
        for card_name in self.card_frequencies:
            freq = self.card_frequencies[card_name]
            for i in range(freq):
                self.cards.append(Card(card_name))
    
    def deal_cards(self):
        for i in range(self.cards_per_player):
            for j in range(len(self.players)):
                player = self.players[j]
                idx = randint(0,len(self.cards)-1)
                player.cards.append(self.cards[idx])
                self.cards.pop(idx)
                
    def deal_card(self, player_idx):
        card_idx = randint(0,len(self.cards)-1)
        self.players[player_idx].cards.append(self.cards[card_idx])
        self.cards.pop(card_idx)
    
    def update_player_turn(self):
        if self.player_turn == len(self.players) - 1:
            self.player_turn = 0
        else:
            self.player_turn += 1
    
    def prompt_player_card(self):
        card = self.players[self.player_turn].prompt_card(self.state)
        return card
    
    def update_state(self, card):
        self.state = PlayAndMeasure.simulate(state=self.state, gate=card.upper())
    
    def update_scores(self):
        if self.state:
            self.players[self.player_turn].score += 1
            status = "win point"
        else:
            self.players[self.player_turn].score -= 1
            status = "lose point"
        
        self.send_player_updates(status)
            
    def update_player_cards(self, player_idx, card_name):
        player = self.players[player_idx]
        for i in range(len(player.cards)):
            card = player.cards[i]
            if card.name == card_name:
                
                player.cards.pop(i)
                return
        print("ERROR: Card {} not found".format(card_name))

    def get_scores(self):
        return [player.score for player in self.players]
        
    def get_scores_update(self):
        scores = self.get_scores()
        updates = ["Current scores"]
        for i in range(len(self.players)):
            player = self.players[i]
            updates.append("{}: {}".format(player.name, scores[i]))
        return updates
    
    
    def send_player_updates(self, status):
        if status in self.player_updates:
            state_update = "The resulting state is {}\n".format(int(self.state))
            update = self.player_updates[status]
            self.players[self.player_turn].update(state_update + update)
            
    def send_players_play_updates(self, player_idx, card_name):
        player = self.players[player_idx]
        result = "won" if self.state else "lost"
        for i in range(len(self.players)):
            if i != player_idx:
                update = "{} played a {} and {} a point.".format(player.name, card_name, result)
                self.players[i].update(update)
    
    def send_players_score_updates(self):
        scores = "\n".join(self.get_scores_update())
        for player in self.players:
            player.update(scores)
                
    def play_a_round(self):
        card_name = self.prompt_player_card().upper()
        
        # qsharp script
        self.update_state(card_name)
        self.update_scores()
        self.update_player_cards(self.player_turn, card_name)
        
        self.send_players_play_updates(self.player_turn, card_name)
        
        if self.check_end_game():
            self.end_game()
        
        self.send_players_score_updates()
        self.deal_card(self.player_turn)
        self.update_player_turn()
        print()
        
    def run(self):
        while not self.check_end_game():
            self.play_a_round()
        self.end_game()
        
    def end_game(self):
        max_score = sorted([player.score for player in self.players])[-1]
        max_players = [player for player in self.players if player.score == max_score]
        if len(max_players) == 1:
            update = "Game Over! {} is the winner. ".format(max_players[0].name)
        else:
            update = "Game Over! {} are the winners. ".format(", ".join([p.name for p in max_players]))
        
        for player in self.players:
            if player.score == max_score:
                update2 = "Congratulations!"
                player.update(update+update2)
            else:
                update2 = "Better luck next time."
                player.update(update+update2)     
            
    def check_end_game(self):
        # TODO
        return len(self.cards) == 0
            

    
    
        

In [170]:
class Player:
    def __init__(self, name):
        self.name = name
        self.score = 0
        self.cards = []
        
    def card_is_valid(self, card_selected):
        for card in self.cards:
            if card.name.lower() == card_selected.lower():
                return True
        return False
        
    def prompt_card(self, current_state):
        card = input("{}, it's your turn. What card would you like to play?\nCurrent state: {}\nYour cards: {}\nYour play: ".format(self.name, int(current_state), [str(card) for card in self.cards]))
        while(not self.card_is_valid(card)):
            card = input("Invalid selection. Please try again.\nCurrent state: {}\nYour cards: {}\n".format(int(current_state),[str(card) for card in self.cards])) ## TODO: give better instructions
        return card
    
    def update(self, string):
        print("[{}] {}".format(self.name,string))
        
    def __str__(self):
        return "{}\nScore: {}\nCards in Hand:{}".format(self.name, self.score, [str(card) for card in self.cards])

    

In [174]:
# Tests
g = Game(card_frequencies={"X": 3, "W": 3, "Y": 3, "Z": 3},
        cards_per_player=4,
        player_names=["arjun","lucy"])
print([str(card) for card in g.cards])
print(g.players[0])
print(g.players[1])
print(g.get_scores())
print()
time.sleep(1)
# g.prompt_player_card()
# g.run()


['X', 'W', 'Y', 'Z']
arjun
Score: 50
Cards in Hand:['Y', 'X', 'W', 'Y']
lucy
Score: 50
Cards in Hand:['Z', 'W', 'X', 'Z']
[50, 50]

